In [1]:
import requests
import json

from collections import Counter

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import cm

import networkx as nx

from Bio import Entrez

In [7]:
df_food = pd.read_csv('/home/sjt/space/github/bioinformatics/data/foodb_2020_04_07_csv/Food.csv')
df_food

,id,name,name_scientific,description,itis_id,wikipedia_id,picture_file_name,picture_content_type,picture_file_size,picture_updated_at,...,food_type,created_at,updated_at,creator_id,updater_id,export_to_afcdb,category,ncbi_taxonomy_id,export_to_foodb,public_id
0,1,Angelica,Angelica keiskei,Angelica is a genus of about 60 species of tal...,NaN,Angelica,1.jpg,image/jpeg,111325.0,2012-04-20 09:29:57 UTC,...,Type 1,2011-02-09 00:37:14 UTC,2019-05-14 18:04:13 UTC,NaN,2.0,False,specific,357850.0,True,FOOD00001
1,2,Savoy cabbage,Brassica oleracea var. sabauda,Savoy cabbage (Brassica oleracea convar. capit...,NaN,Savoy cabbage,2.jpg,image/jpeg,155178.0,2012-04-20 09:39:54 UTC,...,Type 1,2011-02-09 00:37:15 UTC,2019-05-14 18:04:13 UTC,NaN,NaN,False,specific,1216010.0,True,FOOD00002
2,3,Silver linden,Tilia argentea,Tilia tomentosa (Silver Lime in the UK and Sil...,845789.0,Tilia tomentosa,3.jpg,image/jpeg,56367.0,2012-04-20 09:41:25 UTC,...,Type 1,2011-02-09 00:37:15 UTC,2019-05-17 16:19:45 UTC,NaN,NaN,False,specific,NaN,True,FOOD00003
3,4,Kiwi,Actinidia chinensis,"The kiwifruit, often shortened to kiwi in many...",506775.0,Kiwifruit,4.jpg,image/jpeg,110661.0,2012-04-20 09:32:21 UTC,...,Type 1,2011-02-09 00:37:15 UTC,2019-05-14 18:04:13 UTC,NaN,NaN,False,specific,3625.0,True,FOOD00004
4,5,Allium,Allium,Allium haematochiton is a species of wild onio...,42634.0,Allium haematochiton,5.jpg,image/jpeg,341911.0,2012-04-20 09:37:44 UTC,...,Type 1,2011-02-09 00:37:15 UTC,2019-05-14 18:04:13 UTC,NaN,NaN,False,specific,4678.0,True,FOOD00005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,1020,Cape gooseberry,Physalis peruviana,"Physalis peruviana, a plant species of the gen...",NaN,NaN,NaN,NaN,NaN,NaN,...,Type 1,2020-01-24 17:05:37 UTC,2020-01-24 17:05:37 UTC,NaN,NaN,False,NaN,NaN,True,FOOD00988
988,1021,Herbal tea,NaN,Herbal teas are the beverages made from the in...,NaN,NaN,NaN,NaN,NaN,NaN,...,Type 1,2020-03-12 20:55:17 UTC,2020-03-12 20:55:17 UTC,NaN,NaN,False,NaN,NaN,True,FOOD00989
989,1022,Fish oil,NaN,Fish oil is oil derived from the tissues of oi...,NaN,NaN,NaN,NaN,NaN,NaN,...,Type 1,2020-03-12 21:00:02 UTC,2020-03-12 21:00:02 UTC,NaN,NaN,False,NaN,NaN,True,FOOD00990
990,1023,Taco shell,NaN,The hard-shell or crispy taco is a tradition t...,NaN,NaN,NaN,NaN,NaN,NaN,...,Type 2,2020-03-12 21:01:58 UTC,2020-03-12 21:01:58 UTC,NaN,NaN,False,NaN,NaN,True,FOOD00991


In [8]:
df_food.columns

Index(['id', 'name', 'name_scientific', 'description', 'itis_id',
       'wikipedia_id', 'picture_file_name', 'picture_content_type',
       'picture_file_size', 'picture_updated_at', 'legacy_id', 'food_group',
       'food_subgroup', 'food_type', 'created_at', 'updated_at', 'creator_id',
       'updater_id', 'export_to_afcdb', 'category', 'ncbi_taxonomy_id',
       'export_to_foodb', 'public_id'],
      dtype='object')

In [9]:
set(df_food['food_group'])

{'Animal foods',
 'Aquatic foods',
 'Baby foods',
 'Baking goods',
 'Beverages',
 'Cereals and cereal products',
 'Cocoa and cocoa products',
 'Coffee and coffee products',
 'Confectioneries',
 'Dishes',
 'Eggs',
 'Fats and oils',
 'Fruits',
 'Gourds',
 'Herbs and Spices',
 'Herbs and spices',
 'Milk and milk products',
 'Nuts',
 'Pulses',
 'Snack foods',
 'Soy',
 'Teas',
 'Unclassified',
 'Vegetables',
 nan}

In [11]:
m1 = df_food['food_group'] == 'Herbs and Spices'
m2 = df_food['food_group'] == 'Herbs and spices'

m = m1 | m2

df_food[m]

,id,name,name_scientific,description,itis_id,wikipedia_id,picture_file_name,picture_content_type,picture_file_size,picture_updated_at,...,food_type,created_at,updated_at,creator_id,updater_id,export_to_afcdb,category,ncbi_taxonomy_id,export_to_foodb,public_id
0,1,Angelica,Angelica keiskei,Angelica is a genus of about 60 species of tal...,NaN,Angelica,1.jpg,image/jpeg,111325.0,2012-04-20 09:29:57 UTC,...,Type 1,2011-02-09 00:37:14 UTC,2019-05-14 18:04:13 UTC,NaN,2.0,False,specific,357850.0,True,FOOD00001
2,3,Silver linden,Tilia argentea,Tilia tomentosa (Silver Lime in the UK and Sil...,845789.0,Tilia tomentosa,3.jpg,image/jpeg,56367.0,2012-04-20 09:41:25 UTC,...,Type 1,2011-02-09 00:37:15 UTC,2019-05-17 16:19:45 UTC,NaN,NaN,False,specific,NaN,True,FOOD00003
7,8,Garlic,Allium sativum,"With a history of human use of over 7,000 year...",42652.0,Garlic,8.jpg,image/jpeg,545294.0,2012-04-20 09:42:05 UTC,...,Type 1,2011-02-09 00:37:15 UTC,2019-05-14 18:04:13 UTC,NaN,2.0,True,specific,4682.0,True,FOOD00008
8,9,Chives,Allium schoenoprasum,Chives is the common name of Allium schoenopra...,42636.0,Chives,9.jpg,image/jpeg,1411537.0,2012-04-20 09:42:09 UTC,...,Type 1,2011-02-09 00:37:15 UTC,2019-05-14 18:04:13 UTC,NaN,NaN,False,specific,74900.0,True,FOOD00009
9,10,Lemon verbena,Aloysia triphylla,Aloysia citrodora is a species of flowering pl...,32135.0,Aloysia citrodora,10.jpg,image/jpeg,142723.0,2012-04-20 09:32:16 UTC,...,Type 1,2011-02-09 00:37:15 UTC,2019-05-14 18:04:13 UTC,NaN,NaN,False,specific,542672.0,True,FOOD00010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
942,975,Cottonseed oil,NaN,Cottonseed oil is cooking oil from the seeds o...,NaN,NaN,NaN,NaN,NaN,NaN,...,Type 2,2020-01-24 17:05:36 UTC,2020-01-24 17:05:36 UTC,NaN,NaN,False,NaN,NaN,True,FOOD00943
943,976,Corn oil,NaN,Corn oil (maize oil) is oil extracted from the...,NaN,NaN,NaN,NaN,NaN,NaN,...,Type 2,2020-01-24 17:05:36 UTC,2020-01-24 17:05:36 UTC,NaN,NaN,False,NaN,NaN,True,FOOD00944
944,977,Avocado oil,NaN,Avocado oil is an edible oil pressed from the ...,NaN,NaN,NaN,NaN,NaN,NaN,...,Type 2,2020-01-24 17:05:36 UTC,2020-01-24 17:05:36 UTC,NaN,NaN,False,NaN,NaN,True,FOOD00945
945,978,Grapeseed oil,NaN,Grape seed oil (also called grapeseed oil or g...,NaN,NaN,NaN,NaN,NaN,NaN,...,Type 2,2020-01-24 17:05:36 UTC,2020-01-24 17:05:36 UTC,NaN,NaN,False,NaN,NaN,True,FOOD00946
